In [8]:
#importing the modules we need
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [9]:
# the normalize function
def normalize_data(data, type1=None):
    scaler = None

    if type1 == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(data)
        
        scaled_data = scaler.transform(data)
        return scaled_data
    
    elif type1 == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')
        scaler.fit(data)
        
        scaled_data = scaler.transform(data)
        return scaled_data
            
    elif type1 == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')
        scaler.fit(data)
        
        scaled_data = scaler.transform(data)
        return scaled_data

In [10]:
#bag of words class, made during ml lab
class BagOfWords:
    def __init__(self):
        self.vocabulary = dict()
        self.words = []  
        
        
    def build_vocabulary(self, data):
        for sentence in data:
            for word in sentence:
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
                    self.words.append(word)
                    
            
    def get_features(self, data):
        features = np.zeros((len(data), len(self.vocabulary)))
        
        for id_sen, document in enumerate(data):
            for word in document:
                if word in self.vocabulary:
                    features[id_sen, self.vocabulary[word]] += 1
                    
        return features

In [11]:
#getting the data
def load_sample(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    indexes = []
    sentences = []
    
    for line in f.readlines():
        indexes.append(int("".join(line[:6])))
        sentences.append(line[7:].strip('\n').split())
        
    return indexes, sentences


def load_label(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    sentences = []
    
    for line in f.readlines():
        sentences.append(int(line[7]))
        
    return sentences

In [12]:
#train data
train_indexes, train_samples = load_sample("data/train_samples.txt")
train_labels = load_label("data/train_labels.txt")

#validation data
validation_indexes, validation_samples = load_sample("data/validation_samples.txt")
validation_labels = load_label("data/validation_labels.txt")

#test data
test_indexes, test_samples = load_sample("data/test_samples.txt")

In [23]:
#initialize our vocabulary and get the features
bow = BagOfWords()
bow.build_vocabulary(train_samples)

train_features = bow.get_features(train_samples)
validation_features = bow.get_features(validation_samples)
test_features = bow.get_features(test_samples)

In [27]:
#standardize our features
train_features_stand = normalize_data(train_features, type1 = 'l2')
validation_features_stand = normalize_data(validation_features, type1 = 'l2')
test_features_stand = normalize_data(test_features, type1 = 'standard')

In [28]:
#train the model
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(train_features_stand, train_labels)

DecisionTreeClassifier()

In [29]:
#get the prediction on the validation data
predicted = decision_tree_model.predict(validation_features_stand)

print(np.mean(predicted == validation_labels))

0.5896


In [30]:
all_train_samples = train_samples + validation_samples
all_train_labels = train_labels + validation_labels

In [31]:
bow_final = BagOfWords()
bow_final.build_vocabulary(all_train_samples)

all_train_features = bow_final.get_features(all_train_samples)
test_features = bow_final.get_features(test_samples)

In [32]:
#train the model
decision_tree_model_final = DecisionTreeClassifier()
decision_tree_model_final.fit(all_train_features, all_train_labels)

DecisionTreeClassifier()

In [34]:
#get the prediction on the validation data
predicted_final = decision_tree_model_final.predict(test_features)

#and write it in the csv
g = open("data/test_labels.txt", 'w')
g.write("id,label\n")

for idx in range(len(predicted)):
    g.write(f"{test_indexes[idx]},{predicted[idx]}\n")